In [ ]:
import os

In [ ]:
os.environ['KAGGLE_USERNAME'] = 'bluebear78'
os.environ['KAGGLE_KEY'] = '39d305f791ce074d3a7f8d5630fecba2'

In [ ]:
!kaggle competitions download -c digit-recognizer

 65% 10.0M/15.3M [00:00<00:00, 93.8MB/s]
100% 15.3M/15.3M [00:00<00:00, 86.7MB/s]


In [ ]:
!unzip *.*

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt
import numpy as np
from random import randrange
from torch import sigmoid
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

In [ ]:
device = torch.device('cuda')

In [ ]:
df_work = pd.read_csv("./train.csv")

In [ ]:
df_work.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
split_index = round(df_work.shape[0] * 0.9)

In [ ]:
df_train = df_work.iloc[0:split_index, : ]
df_test = df_work.iloc[split_index:,:]

In [ ]:
df_test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
37800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37802,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37803,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37804,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_challenge = pd.read_csv('./test.csv')

In [ ]:
ds_train = torch.utils.data.TensorDataset(torch.tensor(df_train.drop('label', axis=1).values).type(torch.float32), torch.tensor(df_train['label'].values).type(torch.LongTensor))
ds_test = torch.utils.data.TensorDataset(torch.tensor(df_test.drop('label', axis=1).values).type(torch.float32), torch.tensor(df_test['label'].values).type(torch.LongTensor))

In [ ]:
dl_train = torch.utils.data.DataLoader(dataset=ds_train,batch_size=50,shuffle=True)
dl_test = torch.utils.data.DataLoader(dataset=ds_test)

In [ ]:
def show_img(pixels_list):
  fig = px.imshow(torch.tensor(pixels_list).reshape(28,28),binary_format='jpeg',binary_compression_level=0)
  fig.show()

In [ ]:
show_img(df_train.iloc[0,1:].values)

In [ ]:
class CNN_batch(nn.Module):
  def __init__(self,out_1 = 16, out_2 = 32, out_3=64, p=0.2, number_of_classes=10):
    super(CNN_batch,self).__init__()
    self.layer1 = torch.nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=out_1,kernel_size=3,padding=3),
        nn.BatchNorm2d(out_1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout(p=p)
    )
    self.layer2 = torch.nn.Sequential(
        nn.Conv2d(in_channels=out_1,out_channels=out_2,kernel_size=3,stride=1,padding=2),
        nn.BatchNorm2d(out_2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout(p=p)
    )
    self.layer3 = torch.nn.Sequential(
        nn.Conv2d(in_channels=out_2,out_channels=out_3,kernel_size=3,stride=1,padding=4),
        nn.BatchNorm2d(out_3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Dropout(p=p)
    )
    self.fc1 = nn.Linear(out_3*49,number_of_classes)
  
  def forward(self,x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = x.view(x.size(0),-1)
    x = self.fc1(x)
    return x  


In [ ]:
model_batch = CNN_batch(out_1=32,out_2=64,out_3=128)
model_batch = nn.DataParallel(model_batch)#Data distributed Parallel
model_batch.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model_batch.parameters(),lr=learning_rate)

In [ ]:
!pip install pytorchtools 

In [ ]:
def train_model(model,dl_train,dl_test,optimizer,n_epochs=4):
    
    loss_list = []
    accuracy_list = []
    the_last_loss = 100
    patience = 2
    trigger_times = 0
    
    for epoch in range(n_epochs):
        print("Epoch: %s " % str(epoch))
        
        ll_train = []
        for x, y in dl_train:
          x,y = x.to(device),y.to(device)
          model.train()
          optimizer.zero_grad()
#             print(x.reshape(x.shape[0], 1, 28, 28).shape)
            
          yhat = model.forward(x.reshape(x.shape[0], 1, 28, 28))
          loss = criterion(yhat, y)
          loss.backward()
          optimizer.step()
          ll_train.append(loss.data)
            
        l = torch.mean(torch.stack(ll_train)).item()
        loss_list.append(l)
        print('loss data: ', l)
        if l >= the_last_loss:
          trigger_times +=1
          the_last_loss = I
          print('trigger times :',trigger_times)

          if trigger_times >= patience:
            return loss_list, accuracy_list, yhat
          else:
            trigger_times = 0


        correct = 0
        # perform a prediction on the dl_test  
        for x, y in dl_test:
          x,y = x.to(device),y.to(device)
          model.eval()
          z = model(x.reshape(x.shape[0], 1, 28, 28))
          _, yhat = torch.max(z.data, 1)
          correct += (yhat == y).sum().item()
        accuracy = correct / len(dl_test)
        accuracy_list.append(accuracy)
        
    return loss_list, accuracy_list, yhat

In [ ]:
loss_list, accuracy_list, yhat = train_model(model_batch, dl_train, dl_test, optimizer, n_epochs=100) 

Epoch: 0 
loss data:  0.017415042966604233
Epoch: 1 
loss data:  0.018657928332686424
Epoch: 2 
loss data:  0.018614284694194794
Epoch: 3 
loss data:  0.017010308802127838
Epoch: 4 
loss data:  0.018142307177186012
Epoch: 5 
loss data:  0.015442822128534317
Epoch: 6 
loss data:  0.016967423260211945
Epoch: 7 
loss data:  0.016015097498893738
Epoch: 8 
loss data:  0.015275946818292141
Epoch: 9 
loss data:  0.016028940677642822
Epoch: 10 
loss data:  0.014541717246174812
Epoch: 11 
loss data:  0.013506896793842316
Epoch: 12 
loss data:  0.015649067237973213
Epoch: 13 
loss data:  0.013250116258859634
Epoch: 14 
loss data:  0.014611621387302876
Epoch: 15 
loss data:  0.01471760030835867
Epoch: 16 
loss data:  0.012935600243508816
Epoch: 17 
loss data:  0.014186578802764416
Epoch: 18 
loss data:  0.011758310720324516
Epoch: 19 
loss data:  0.013964422978460789
Epoch: 20 
loss data:  0.012106256559491158
Epoch: 21 
loss data:  0.012989909388124943
Epoch: 22 
loss data:  0.013331296853721142

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=list(range(0, len(loss_list))), y=loss_list, name="loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=list(range(0, len(loss_list))), y=accuracy_list, name="accuracy"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Loss and Accuracy"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Loss</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Accuracy</b>", secondary_y=True)

fig.show()

In [ ]:
ds_challenge = torch.utils.data.TensorDataset(torch.tensor(df_challenge.values).type(torch.float32))
dl_challenge = torch.utils.data.DataLoader(dataset=ds_challenge)

df_submission  = pd.read_csv('./sample_submission.csv')

In [ ]:
print(df_submission.shape, len(dl_challenge))
df_submission.head()

(28000, 2) 28000


,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
for i, x in enumerate(dl_challenge):
    x = x[0]
    x = x.to(device)
    z = model_batch(x.reshape(x.shape[0], 1, 28, 28))
    _, yhat = torch.max(z.data, 1)
    df_submission.at[i, 'Label'] = yhat.item()

In [ ]:
df_submission.to_csv('./my_submission.csv')

In [ ]:
!kaggle competitions submit -c digit-recognizer -f my_submission.csv -m "Message"

100% 361k/361k [00:01<00:00, 208kB/s]
Successfully submitted to Digit Recognizer